In [ ]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime


class PawnShopDB:
    def __init__(self, db_name='pawnshop_db.db'):
        self.conn = sqlite3.connect(db_name)
        self.create_table()

    def create_table(self):
        self.conn.execute('''
            CREATE TABLE IF NOT EXISTS items (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer TEXT UNIQUE,
                item TEXT,
                value REAL,
                date TEXT,
                UNIQUE(customer, item)
            )
        ''')
        self.conn.commit()

    def add_item(self, customer, item, value, date):
        try:
            self.conn.execute(
                'INSERT INTO items (customer, item, value, date) VALUES (?, ?, ?, ?)',
                (customer, item, value, date)
            )
            self.conn.commit()
            return True, "Item added successfully"
        except sqlite3.IntegrityError:
            return False, "Error: This item is already pawned by this customer"

    def get_items(self):
        return self.conn.execute('SELECT * FROM items').fetchall()

    def search_items(self, customer="", date=""):
        query = 'SELECT * FROM items WHERE 1=1'
        params = []
        if customer:
            query += ' AND customer LIKE ?'
            params.append(f'%{customer}%')
        if date:
            query += ' AND date = ?'
            params.append(date)
        return self.conn.execute(query, params).fetchall()

    def update_item(self, item_id, customer, item, value, date):
        try:
            self.conn.execute(
                'UPDATE items SET customer=?, item=?, value=?, date=? WHERE id=?',
                (customer, item, value, date, item_id)
            )
            self.conn.commit()
            return True, "Item updated successfully"
        except sqlite3.IntegrityError:
            return False, "Error: This combination of customer and item already exists"

    def delete_item(self, item_id):
        self.conn.execute('DELETE FROM items WHERE id = ?', (item_id,))
        self.conn.commit()

    def get_item_by_id(self, item_id):
        return self.conn.execute('SELECT * FROM items WHERE id = ?', (item_id,)).fetchone()

class PawnShopApp:
    def __init__(self):
        self.db = PawnShopDB()
        self.editing_id = None
        self.build_ui()

    def build_ui(self):
        self.name_input = widgets.Text(description="Customer:")
        self.item_input = widgets.Text(description="Item:")
        self.value_input = widgets.FloatText(description="Value:")
        self.date_input = widgets.DatePicker(description="Date:")
        self.add_button = widgets.Button(description="Add Item", button_style='success')
        self.update_button = widgets.Button(description="Update Item", button_style='info')
        self.update_button.layout.display = 'none'
        self.cancel_button = widgets.Button(description="Cancel", button_style='warning')
        self.cancel_button.layout.display = 'none'
        self.search_customer = widgets.Text(description="Customer:", placeholder="Search by customer name")
        self.search_date = widgets.DatePicker(description="Date:")
        self.search_button = widgets.Button(description="Search", button_style='primary')
        self.clear_search_button = widgets.Button(description="Clear")
        self.output = widgets.Output()
        self.item_box = widgets.VBox()
        self.add_button.on_click(self.add_item)
        self.update_button.on_click(self.update_item)
        self.cancel_button.on_click(self.cancel_edit)
        self.search_button.on_click(self.search_items)
        self.clear_search_button.on_click(self.clear_search)
        form = widgets.VBox([
            self.name_input,
            self.item_input,
            self.value_input,
            self.date_input,
            widgets.HBox([self.add_button, self.update_button, self.cancel_button])
        ], layout=widgets.Layout(width='50%', padding='10px'))
        search_form = widgets.VBox([
            widgets.HTML("<h3>Search Items</h3>"),
            self.search_customer,
            self.search_date,
            widgets.HBox([self.search_button, self.clear_search_button])
        ], layout=widgets.Layout(width='50%', padding='10px'))
        left_panel = widgets.VBox([form, search_form])
        app_layout = widgets.HBox([left_panel, self.item_box])
        display(app_layout, self.output)
        self.refresh_items()

    def add_item(self, b):
        name = self.name_input.value
        item = self.item_input.value
        value = self.value_input.value
        date = str(self.date_input.value) if self.date_input.value else ""
        if name and item and value and date:
            success, message = self.db.add_item(name, item, value, date)
            with self.output:
                clear_output()
                if success:
                    print(message)
                    self.name_input.value = ""
                    self.item_input.value = ""
                    self.value_input.value = 0.0
                    self.date_input.value = None
                else:
                    print(message)
            self.refresh_items()
        else:
            with self.output:
                clear_output()
                print("Error: Please fill all fields.")

    def edit_item(self, item_id):
        item_data = self.db.get_item_by_id(item_id)
        if item_data:
            self.editing_id = item_id
            self.name_input.value = item_data[1]
            self.item_input.value = item_data[2]
            self.value_input.value = item_data[3]
            self.date_input.value = datetime.strptime(item_data[4], '%Y-%m-%d').date()
            self.add_button.layout.display = 'none'
            self.update_button.layout.display = 'block'
            self.cancel_button.layout.display = 'block'

    def update_item(self, b):
        if not self.editing_id:
            return
        name = self.name_input.value
        item = self.item_input.value
        value = self.value_input.value
        date = str(self.date_input.value) if self.date_input.value else ""
        if name and item and value and date:
            success, message = self.db.update_item(self.editing_id, name, item, value, date)
            with self.output:
                clear_output()
                print(message)
            if success:
                self.cancel_edit(None)
            self.refresh_items()
        else:
            with self.output:
                clear_output()
                print("Error: Please fill all fields.")

    def cancel_edit(self, b):
        self.editing_id = None
        self.name_input.value = ""
        self.item_input.value = ""
        self.value_input.value = 0.0
        self.date_input.value = None
        self.add_button.layout.display = 'block'
        self.update_button.layout.display = 'none'
        self.cancel_button.layout.display = 'none'

    def search_items(self, b):
        customer = self.search_customer.value
        date = str(self.search_date.value) if self.search_date.value else ""
        if not customer and not date:
            with self.output:
                clear_output()
                print("Please enter at least one search criteria")
            return
        items = self.db.search_items(customer, date)
        self.display_items(items)
        with self.output:
            clear_output()
            print(f"Found {len(items)} items matching your search criteria")

    def clear_search(self, b):
        self.search_customer.value = ""
        self.search_date.value = None
        self.refresh_items()
        with self.output:
            clear_output()
            print("Search cleared")

    def refresh_items(self):
        items = self.db.get_items()
        self.display_items(items)

    def display_items(self, items):
        children = []
        if not items:
            no_items = widgets.HTML("<p><i>No items found</i></p>")
            children.append(no_items)
        else:
            header = widgets.HTML(
                "<div style='font-weight: bold; margin-bottom: 10px;'>" +
                "<span style='display: inline-block; width: 150px;'>Customer</span>" +
                "<span style='display: inline-block; width: 150px;'>Item</span>" +
                "<span style='display: inline-block; width: 100px;'>Value</span>" +
                "<span style='display: inline-block; width: 100px;'>Date</span>" +
                "</div>"
            )
            children.append(header)
            for id_, customer, item, value, date in items:
                info = widgets.HTML(
                    f"<div>" +
                    f"<span style='display: inline-block; width: 150px;'>{customer}</span>" +
                    f"<span style='display: inline-block; width: 150px;'>{item}</span>" +
                    f"<span style='display: inline-block; width: 100px;'>${value:.2f}</span>" +
                    f"<span style='display: inline-block; width: 100px;'>{date}</span>" +
                    f"</div>"
                )
                edit_btn = widgets.Button(description="Edit", button_style='info', layout=widgets.Layout(width='60px'))
                edit_btn.on_click(lambda b, item_id=id_: self.edit_item(item_id))
                del_btn = widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='80px'))
                del_btn.on_click(lambda b, item_id=id_: self.delete_item(item_id))
                row = widgets.HBox([info, edit_btn, del_btn])
                children.append(row)
        self.item_box.children = children

    def delete_item(self, item_id):
        self.db.delete_item(item_id)
        self.refresh_items()
        with self.output:
            clear_output()
            print("Item deleted successfully")


pawnShopApp = PawnShopApp()

Output()